# Social Media Group Project: Subreddit Homophily and Influence

## 1. Initialize

### 1.1 Import necessary libraries

In [48]:

import pandas as pd
import zstandard as zstd
import matplotlib.pyplot as plt
import os


### 1.2 Read compressed JSON files and read into list of dataframes

In [49]:
def read_json_with_compression(filepath):
    try:
        # Decompressing the zstd file to JSON
        with open(filepath, 'rb') as compressed:
            dctx = zstd.ZstdDecompressor()
            json_output_filepath = os.path.splitext(filepath)[0] + '.json'
            with open(json_output_filepath, 'wb') as decompressed:
                dctx.copy_stream(compressed, decompressed)
        # Reading JSON into DataFrame
        df = pd.read_json(json_output_filepath, lines=True)
        print(f"Data from {filepath} loaded successfully!")
        return df
    except ValueError as e:
        print(f"Error reading the JSON file at {filepath}: {e}")
    except Exception as e:
        print(f"An error occurred with file at {filepath}: {e}")
    return None

# Define file paths for each data set
filepaths = {
    'dog_comments': "DOG_comments.zst",
    'dog_submissions': "DOG_submissions.zst",
    'pets_comments': "Pets_comments.zst",
    'pets_submissions':"Pets_submissions.zst",
    'puppies_comments': "puppies_comments.zst",
    'puppies_submissions': "puppies_submissions.zst",
    'puppy_smiles_comments': "PuppySmiles_comments.zst",
    'puppy_smiles_submissions': "PuppySmiles_submissions.zst"
}

dataframes = {}

# Read data into dataframes
for key, filepath in filepaths.items():
    dataframes[key] = read_json_with_compression(filepath)

# Access dataframes using keys
df_dog_comments = dataframes['dog_comments']
df_dog_submissions = dataframes['dog_submissions']
df_pets_comments = dataframes['pets_comments']
df_pets_submissions = dataframes['pets_submissions']
df_puppies_comments = dataframes['puppies_comments']
df_puppies_submissions = dataframes['puppies_submissions']
df_puppy_smiles_comments = dataframes['puppy_smiles_comments']
df_puppy_smiles_submissions = dataframes['puppy_smiles_submissions']


Data from DOG_comments.zst loaded successfully!
Data from DOG_submissions.zst loaded successfully!
Data from Pets_comments.zst loaded successfully!
Data from Pets_submissions.zst loaded successfully!
Data from puppies_comments.zst loaded successfully!
Data from puppies_submissions.zst loaded successfully!
Data from PuppySmiles_comments.zst loaded successfully!
Data from PuppySmiles_submissions.zst loaded successfully!


## 2. Preprocessing

### 2.1 Check for missing ID's and define set of dataframes

In [50]:
# List of DataFrames
dataframes = {
    'dog_comments': df_dog_comments,
    'dog_submissions': df_dog_submissions,
    'pets_comments': df_pets_comments,
    'pets_submissions': df_pets_submissions,
    'puppies_comments': df_puppies_comments,
    'puppies_submissions': df_puppies_submissions,
    'puppy_smiles_comments': df_puppy_smiles_comments,
    'puppy_smiles_submissions': df_puppy_smiles_submissions
}

# Check for missing 'id' in each DataFrame
missing_ids = {}
for name, df in dataframes.items():
    missing_ids[name] = df[df['id'].isnull()]

# Print results
for name, df_missing_ids in missing_ids.items():
    print(f"Missing 'id' in {name}:")
    print(df_missing_ids)
    print("\n")

# List of DataFrames for comments
dataframes_comments = {
    'dog_comments': df_dog_comments,
    'pets_comments': df_pets_comments,
    'puppies_comments': df_puppies_comments,
    'puppy_smiles_comments': df_puppy_smiles_comments
}

# Check for missing 'link_id' in each DataFrame
missing_ids_comments = {}
for name, df in dataframes_comments.items():
    missing_ids_comments[name] = df[df['link_id'].isnull()]

# Print results
for name, df_missing_ids in missing_ids_comments.items():
    print(f"Missing 'link_id' in {name}:")
    print(df_missing_ids)
    print("\n")

# Access dataframes using keys and get the head
df_dog_comments = dataframes['dog_comments']
print("Head of df_dog_comments:")
print(df_dog_comments.head())

df_dog_submissions = dataframes['dog_submissions']
print("\nHead of df_dog_submissions:")
print(df_dog_submissions.head())

df_pets_comments = dataframes['pets_comments']
print("\nHead of df_pets_comments:")
print(df_pets_comments.head())

df_pets_submissions = dataframes['pets_submissions']
print("\nHead of df_pets_submissions:")
print(df_pets_submissions.head())

df_puppies_comments = dataframes['puppies_comments']
print("\nHead of df_puppies_comments:")
print(df_puppies_comments.head())

df_puppies_submissions = dataframes['puppies_submissions']
print("\nHead of df_puppies_submissions:")
print(df_puppies_submissions.head())

df_puppy_smiles_comments = dataframes['puppy_smiles_comments']
print("\nHead of df_puppy_smiles_comments:")
print(df_puppy_smiles_comments.head())

df_puppy_smiles_submissions = dataframes['puppy_smiles_submissions']
print("\nHead of df_puppy_smiles_submissions:")
print(df_puppy_smiles_submissions.head())

# List of DataFrames (updated)
dataframes = {
    'dog_comments': df_dog_comments,
    'dog_submissions': df_dog_submissions,
    'pets_comments': df_pets_comments,
    'pets_submissions': df_pets_submissions,
    'puppies_comments': df_puppies_comments,
    'puppies_submissions': df_puppies_submissions,
    'puppy_smiles_comments': df_puppy_smiles_comments,
    'puppy_smiles_submissions': df_puppy_smiles_submissions
}


Missing 'id' in dog_comments:
Empty DataFrame
Columns: [score_hidden, created_utc, name, ups, author_flair_text, edited, body, archived, parent_id, score, distinguished, controversiality, author_flair_css_class, gilded, link_id, retrieved_on, id, downs, subreddit_id, author, subreddit, removal_reason, stickied, can_gild, author_cakeday, approved_at_utc, can_mod_post, banned_at_utc, collapsed, collapsed_reason, is_submitter, permalink, subreddit_type, mod_note, mod_reason_by, mod_reason_title, no_follow, send_replies, author_flair_template_id, approved_by, banned_by, body_html, likes, mod_reports, num_reports, replies, report_reasons, saved, user_reports, author_flair_background_color, author_flair_richtext, author_flair_text_color, author_flair_type, rte_mode, author_created_utc, author_fullname, subreddit_name_prefixed, gildings, author_patreon_flair, quarantined, locked, all_awardings, total_awards_received, steward_reports, awarders, associated_award, collapsed_because_crowd_control

### 2.2 Convert unix timestamp to datetime

In [51]:
# Convert 'created_utc' from UNIX timestamp to datetime for each dataframe
for df_name, df in dataframes.items():
    df['created_datetime'] = pd.to_datetime(df['created_utc'], unit='s')

# Initialize an empty set to store unique years
distinct_years = set()

# Iterate through each DataFrame and extract unique years
for df_name, df in dataframes.items():
    distinct_years.update(df['created_datetime'].dt.year.unique())

# Print the number of distinct years
print("Number of distinct years:", len(distinct_years))

# Print the distinct years found
print("Distinct years:", sorted(distinct_years))

Number of distinct years: 15
Distinct years: [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]


### 2.3 Filter data by base year (2017)

In [52]:
# Define base year
base_year = 2017

# Filter each dataframe to include only the base year
filtered_dataframes = {}
for df_name, df in dataframes.items():
    filtered_dataframes[df_name] = df[df['created_datetime'].dt.year == base_year]

# Filter out rows with author as '[deleted]' for each DataFrame
filtered_dataframes_no_deleted = {}
for name, df in filtered_dataframes.items():
    filtered_dataframes_no_deleted[name] = df[df['author'] != '[deleted]']

# Access filtered DataFrames, create new ones with the year in front of the name
df_base_year_filtered_dataframes = {}
for name, df in filtered_dataframes_no_deleted.items():
    df_base_year_filtered_dataframes[f"df_{base_year}_{name}_filtered"] = df

# Access filtered DataFrames
for name, df in df_base_year_filtered_dataframes.items():
    print(f"{name} = df_base_year_filtered_dataframes['{name}']")


df_2017_dog_comments_filtered = df_base_year_filtered_dataframes['df_2017_dog_comments_filtered']
df_2017_dog_submissions_filtered = df_base_year_filtered_dataframes['df_2017_dog_submissions_filtered']
df_2017_pets_comments_filtered = df_base_year_filtered_dataframes['df_2017_pets_comments_filtered']
df_2017_pets_submissions_filtered = df_base_year_filtered_dataframes['df_2017_pets_submissions_filtered']
df_2017_puppies_comments_filtered = df_base_year_filtered_dataframes['df_2017_puppies_comments_filtered']
df_2017_puppies_submissions_filtered = df_base_year_filtered_dataframes['df_2017_puppies_submissions_filtered']
df_2017_puppy_smiles_comments_filtered = df_base_year_filtered_dataframes['df_2017_puppy_smiles_comments_filtered']
df_2017_puppy_smiles_submissions_filtered = df_base_year_filtered_dataframes['df_2017_puppy_smiles_submissions_filtered']


### 2.4 Filter data by comparison year (2019)

In [53]:
# Define comparison year
comparison_year = 2019

# Filter each dataframe to include only the comparison year
filtered_dataframes = {}
for df_name, df in dataframes.items():
    filtered_dataframes[df_name] = df[df['created_datetime'].dt.year == comparison_year]

# Filter out rows with author as '[deleted]' for each DataFrame
filtered_dataframes_no_deleted = {}
for name, df in filtered_dataframes.items():
    filtered_dataframes_no_deleted[name] = df[df['author'] != '[deleted]']

# Access filtered DataFrames, create new ones with the year in front of the name
df_comparison_year_filtered_dataframes = {}
for name, df in filtered_dataframes_no_deleted.items():
    df_comparison_year_filtered_dataframes[f"df_{comparison_year}_{name}_filtered"] = df

# Access filtered DataFrames
for name, df in df_comparison_year_filtered_dataframes.items():
    print(f"{name} = df_comparison_year_filtered_dataframes['{name}']")


df_2019_dog_comments_filtered = df_comparison_year_filtered_dataframes['df_2019_dog_comments_filtered']
df_2019_dog_submissions_filtered = df_comparison_year_filtered_dataframes['df_2019_dog_submissions_filtered']
df_2019_pets_comments_filtered = df_comparison_year_filtered_dataframes['df_2019_pets_comments_filtered']
df_2019_pets_submissions_filtered = df_comparison_year_filtered_dataframes['df_2019_pets_submissions_filtered']
df_2019_puppies_comments_filtered = df_comparison_year_filtered_dataframes['df_2019_puppies_comments_filtered']
df_2019_puppies_submissions_filtered = df_comparison_year_filtered_dataframes['df_2019_puppies_submissions_filtered']
df_2019_puppy_smiles_comments_filtered = df_comparison_year_filtered_dataframes['df_2019_puppy_smiles_comments_filtered']
df_2019_puppy_smiles_submissions_filtered = df_comparison_year_filtered_dataframes['df_2019_puppy_smiles_submissions_filtered']


## 3. Activity Analysis and Top User Identification

This section of code performs an analysis of user activity based on submissions and comments for a specific year. It calculates the total activity for each user by summing the number of posts and comments, then identifies the top 20 most active users. Finally, it generates a combination DataFrame of the top users for further analysis.

The combination DataFrame generated in the provided code represents all possible pairs of users among the top 20 most active users. Each row in the DataFrame contains a pair of users where one user is considered the "Parent ID" and the other user is considered the "Child ID".

In [54]:
# Group by 'author' and 'year' to count the number of submissions
submission_counts = df_base_year_filtered_dataframes[f"df_{base_year}_dog_submissions_filtered"].groupby(
    ['author', df_base_year_filtered_dataframes[f"df_{base_year}_dog_submissions_filtered"]['created_datetime'].dt.year]
).size().reset_index(name='post_count')

# Group by 'author' and 'year' to count the number of comments
comment_counts = df_base_year_filtered_dataframes[f"df_{base_year}_dog_comments_filtered"].groupby(
    ['author', df_base_year_filtered_dataframes[f"df_{base_year}_dog_comments_filtered"]['created_datetime'].dt.year]
).size().reset_index(name='comment_count')

# Merge the two datasets on 'author' and 'year' columns
total_activity = pd.merge(
    submission_counts,
    comment_counts,
    on=['author', 'created_datetime'],  # Adjust to merge on the correct column names if 'created_datetime' stores the year
    how='outer'
)

# Replace NaN values with 0 in case there are authors who only posted or only commented
total_activity.fillna(0, inplace=True)

# Calculate the total activity for each user by summing posts and comments
total_activity['total_activity'] = total_activity['post_count'] + total_activity['comment_count']

# Sort the dataframe by 'total_activity' in descending order to get the most active users
top_users = total_activity.sort_values(by='total_activity', ascending=False)

# Print the top 20 most active users
print(top_users.head(20))

# Select all the top users
top_20_users = top_users
top_20_users_dog_2017 = top_20_users['author']
top_20_combination_df = pd.DataFrame([(x, y) for x in top_20_users_dog_2017  for y in top_20_users_dog_2017  if x != y], columns=["Parent ID", "Child ID"])


                author  created_datetime  post_count  comment_count  \
689        Tillmantino              2017       334.0            0.0   
1805     AutoModerator              2017         0.0          154.0   
2             0010pond              2017        53.0            0.0   
1997     imguralbumbot              2017         0.0           45.0   
941      crazydogsinfo              2017        30.0            0.0   
504      NowRecyclable              2017        11.0           18.0   
559       Puppuniverse              2017        26.0            0.0   
1779            zanliu              2017        20.0            4.0   
1641  squirrelinstinct              2017         3.0           17.0   
805            antdude              2017        20.0            0.0   
576            Repreve              2017        11.0            7.0   
1801          Archaris              2017         0.0           18.0   
785        allenmonkey              2017        14.0            0.0   
804   

## 4. Reddit User Interaction Analysis

### 4.1 Function: Process and Merge Comments and Submissions

In [55]:
def process_and_merge_comments_submissions(comments_df, submissions_df):
    # Extract year from 'created_datetime' column
    comments_df['Year'] = pd.to_datetime(comments_df['created_datetime']).dt.year
    submissions_df['Year'] = pd.to_datetime(submissions_df['created_datetime']).dt.year
    
    comments_df['noprefix_parent_id'] = comments_df['parent_id'].str[3:]
    
    mask_t3 = comments_df['parent_id'].str.startswith('t3')
    df_t3 = comments_df[mask_t3].copy()
    df_t3_merged = df_t3.merge(submissions_df[['id', 'author', 'Year']], left_on='noprefix_parent_id', right_on='id', suffixes=('_child', '_parent'))
    df_t3_merged['Link Type'] = 'respond to a submission'

    mask_t1 = comments_df['parent_id'].str.startswith('t1')
    df_t1 = comments_df[mask_t1].copy()
    df_t1_merged = df_t1.merge(comments_df[['id', 'author', 'Year']], left_on='noprefix_parent_id', right_on='id', suffixes=('_child', '_parent'))
    df_t1_merged['Link Type'] = 'respond to a comment'

    # Combine both sets of merged data
    combined_df = pd.concat([df_t3_merged, df_t1_merged], ignore_index=True)
    
    # Choose the earliest year if there's a discrepancy between child and parent
    combined_df['Interaction Year'] = combined_df[['Year_child', 'Year_parent']].min(axis=1)
    
    # Clean up the DataFrame
    combined_df = combined_df.drop(columns=['Year_child', 'Year_parent'])
    
    return combined_df

### 4.2 Processing and Merging Comments and Submissions

In [56]:
final_dataframes = {}

# List of subreddits
subreddits = ['dog', 'pets', 'puppies', 'puppy_smiles']

for subreddit in subreddits:
    comments_df = df_base_year_filtered_dataframes[f'df_2017_{subreddit}_comments_filtered']
    submissions_df = df_base_year_filtered_dataframes[f'df_2017_{subreddit}_submissions_filtered']
    final_dataframes[subreddit] = process_and_merge_comments_submissions(comments_df, submissions_df)

# After processing, update the dataframes with the new structure including 'Interaction Year'
for subreddit, df in final_dataframes.items():
    df = df[['author_child', 'author_parent', 'Link Type', 'Interaction Year']]
    df.columns = ['Child ID', 'Parent ID', 'Link Type', 'Year']
    final_dataframes[subreddit] = df

dog = final_dataframes['dog']

### 4.3 Generating Tie Tables for 2017 and 2019

In [57]:
merged_2017 = top_20_combination_df.merge(dog, on=['Parent ID', 'Child ID'], how='left')
merged_2017['binary indicator'] = merged_2017['Link Type'].notnull().astype(int)
merged_2017 = merged_2017[['Parent ID', 'Child ID', 'binary indicator']]

import numpy as np
# Normalize the order of Parent ID and Child ID
merged_2017[['Parent ID', 'Child ID']] = pd.DataFrame(np.sort(merged_2017[['Parent ID', 'Child ID']].values, axis=1), index=merged_2017.index)

# Group by Parent ID and Child ID and aggregate the binary indicator
merged_2017_group = merged_2017.groupby(['Parent ID', 'Child ID'], as_index=False)['binary indicator'].max()
merged_2017_group.rename(columns={'binary indicator': '2017 Tie'}, inplace=True)
merged_2017_group

final_dataframes_2019 = {}

# List of subreddits
subreddits = ['dog', 'pets', 'puppies', 'puppy_smiles']

for subreddit in subreddits:
    comments_df = df_comparison_year_filtered_dataframes[f'df_2019_{subreddit}_comments_filtered']
    submissions_df = df_comparison_year_filtered_dataframes[f'df_2019_{subreddit}_submissions_filtered']
    final_dataframes_2019[subreddit] = process_and_merge_comments_submissions(comments_df, submissions_df)

# After processing, update the dataframes with the new structure including 'Interaction Year'
for subreddit, df in final_dataframes_2019.items():
    df = df[['author_child', 'author_parent', 'Link Type', 'Interaction Year']]
    df.columns = ['Child ID', 'Parent ID', 'Link Type', 'Year']
    final_dataframes_2019[subreddit] = df

dog_2019 = final_dataframes_2019['dog']
dog_2019

merged_2019 = top_20_combination_df.merge(dog_2019, on=['Parent ID', 'Child ID'], how='left')
merged_2019['binary indicator'] = merged_2019['Link Type'].notnull().astype(int)
merged_2019 = merged_2019[['Parent ID', 'Child ID', 'binary indicator']]

import numpy as np
# Normalize the order of Parent ID and Child ID
merged_2019[['Parent ID', 'Child ID']] = pd.DataFrame(np.sort(merged_2019[['Parent ID', 'Child ID']].values, axis=1), index=merged_2019.index)

# Group by Parent ID and Child ID and aggregate the binary indicator
merged_2019_group = merged_2019.groupby(['Parent ID', 'Child ID'], as_index=False)['binary indicator'].max()
merged_2019_group.rename(columns={'binary indicator': '2019 Tie'}, inplace=True)
merged_2019_group

tie_table = merged_2017_group.merge(merged_2019_group, on=['Parent ID', 'Child ID'], how='left')
tie_table


C:\Users\seanc\AppData\Local\Temp\ipykernel_16828\2870910691.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comments_df['Year'] = pd.to_datetime(comments_df['created_datetime']).dt.year
C:\Users\seanc\AppData\Local\Temp\ipykernel_16828\2870910691.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submissions_df['Year'] = pd.to_datetime(submissions_df['created_datetime']).dt.year
C:\Users\seanc\AppData\Local\Temp\ipykernel_16828\2870910691.py:6: SettingWithCopyWarning: 
A value is trying to be set on a

,Parent ID,Child ID,2017 Tie,2019 Tie
0,--i_make_things--,-avi1998,0,0
1,--i_make_things--,-jinacio-,0,0
2,--i_make_things--,0010110012Amen,0,0
3,--i_make_things--,0010pond,0,0
4,--i_make_things--,02tylerrobert,0,0
...,...,...,...,...
2191366,zinzzzin,zoepeterson02,0,0
2191367,zinzzzin,zww8169,0,0
2191368,zlifea,zoepeterson02,0,0
2191369,zlifea,zww8169,0,0


### 4.4 Feature Table Generation

In [58]:
Top_20_users_against_baseyear = top_20_users.copy()  # Create a copy of top_20_users

# Initialize new columns with default value 0
for df_name, df in df_base_year_filtered_dataframes.items():
    if df_name not in [f"df_{base_year}_dog_comments_filtered", f"df_{base_year}_dog_submissions_filtered"]:
         Top_20_users_against_baseyear[f'in_{df_name}'] = 0

# Iterate through each user in top_20_users dataframe
for index, row in top_20_users.iterrows():
    author = row['author']
    
    # Check if the user is present in each dataframe
    for df_name, df in df_base_year_filtered_dataframes.items():
        if author in df['author'].values:
            Top_20_users_against_baseyear.at[index, f'in_{df_name}'] = 1


Top_20_users_against_baseyear['General Interest 2017'] = (Top_20_users_against_baseyear.iloc[:, 5:-2].any(axis=1)).astype(int)
Top_20_users_against_comparisonyear = top_20_users.copy()  # Create a copy of top_20_users

# Initialize new columns with default value 0
for df_name, df in df_comparison_year_filtered_dataframes.items():
         Top_20_users_against_comparisonyear[f'in_{df_name}'] = 0

# Iterate through each user in top_20_users dataframe
for index, row in top_20_users.iterrows():
    author = row['author']
    
    # Check if the user is present in each dataframe
    for df_name, df in df_comparison_year_filtered_dataframes.items():
        if author in df['author'].values:
            Top_20_users_against_comparisonyear.at[index, f'in_{df_name}'] = 1

Top_20_users_against_comparisonyear['General Interest 2019'] = (Top_20_users_against_comparisonyear.iloc[:, 7:].any(axis=1)).astype(int)
feature_table = Top_20_users_against_baseyear.merge(Top_20_users_against_comparisonyear, on='author', how='outer', suffixes=('_2017', '_2019'))
feature_table = feature_table[['author', 'General Interest 2017', 'General Interest 2019']]

feature_table

,author,General Interest 2017,General Interest 2019
0,Tillmantino,0,0
1,AutoModerator,0,0
2,0010pond,0,0
3,imguralbumbot,1,1
4,crazydogsinfo,0,0
...,...,...,...
2089,codepro1989,0,0
2090,cocols,0,0
2091,cnuss1,0,0
2092,Ciliu,0,0


## 5. User Interaction Analysis: Chi-Square Test for Association

This section of code conducts a chi-square test for association to analyze the relationship between the general interests of users in 2017 and 2019, considering the ties between users as well.



### 5.1 Create combination dataframe

In [59]:
from itertools import combinations

# Create combinations DataFrame
combinations_df = pd.DataFrame(list(combinations(feature_table['author'], 2)), columns=['Parent ID', 'Child ID'])


### 5.2 Merge to Find **Shared** General Interests and Map Them

In [60]:
# Merge to find General Interests for both 'Parent ID' and 'Child ID'
combi_table = combinations_df.merge(feature_table, left_on='Parent ID', right_on='author')
combi_table = combi_table.merge(feature_table, left_on='Child ID', right_on='author', suffixes=('_Parent', '_Child'))

# Calculate the sum of 'General Interest 2017' and 'General Interest 2019' for the new table
combi_table['General Interest 2017'] = combi_table['General Interest 2017_Parent'] + combi_table['General Interest 2017_Child']
combi_table['General Interest 2019'] = combi_table['General Interest 2019_Parent'] + combi_table['General Interest 2019_Child']


# Function to map sums to 0 or 1 based on the described logic
def map_interest(value):
    return 1 if value in [0, 2] else 0

# Apply the mapping function to the General Interest columns
combi_table['General Interest 2017'] = combi_table['General Interest 2017'].apply(map_interest)
combi_table['General Interest 2019'] = combi_table['General Interest 2019'].apply(map_interest)


### 5.3 Dataframe cleanup and final prep for analysis

In [61]:
# Drop unnecessary columns
combi_table = combi_table[['Parent ID', 'Child ID', 'General Interest 2017', 'General Interest 2019']]

# Normalize 'Parent ID' and 'Child ID' order in both dataframes
combi_table['sorted_id_1'] = combi_table.apply(lambda x: sorted([x['Parent ID'], x['Child ID']])[0], axis=1)
combi_table['sorted_id_2'] = combi_table.apply(lambda x: sorted([x['Parent ID'], x['Child ID']])[1], axis=1)
tie_table['sorted_id_1'] = tie_table.apply(lambda x: sorted([x['Parent ID'], x['Child ID']])[0], axis=1)
tie_table['sorted_id_2'] = tie_table.apply(lambda x: sorted([x['Parent ID'], x['Child ID']])[1], axis=1)

# Merge the tables using the sorted ID columns
merged_table = combi_table.merge(tie_table, left_on=['sorted_id_1', 'sorted_id_2'], right_on=['sorted_id_1', 'sorted_id_2'], how='left')

# Clean up: Drop the temporary sorted ID columns if no longer needed
merged_table.drop(columns=['sorted_id_1', 'sorted_id_2'], inplace=True)

# Handle missing values from the merge, if any (assuming missing ties are 0)
merged_table.fillna({'2017 Tie': 0, '2019 Tie': 0}, inplace=True)


### 5.4 Chi-Square Tests and Results

In [62]:
# Analyze Results with Chi-Square Test
# Value counts of 'General Interest 2017' when there is a tie in 2017
print("Value counts of 'General Interest 2017' with tie in 2017:\n", merged_table[merged_table['2017 Tie'] == 1]['General Interest 2017'].value_counts())

# Value counts of 'General Interest 2017' when there is no tie in 2017
print("Value counts of 'General Interest 2017' without tie in 2017:\n", merged_table[merged_table['2017 Tie'] == 0]['General Interest 2017'].value_counts())

# Value counts of 'General Interest 2017' when there is a tie in 2019
print("Value counts of 'General Interest 2017' with tie in 2019:\n", merged_table[merged_table['2019 Tie'] == 1]['General Interest 2017'].value_counts())

# Value counts of 'General Interest 2017' when there is no tie in 2019
print("Value counts of 'General Interest 2017' without tie in 2019:\n", merged_table[merged_table['2019 Tie'] == 0]['General Interest 2017'].value_counts())

# Value counts of 'General Interest 2019' when there is a tie in 2017
print("Value counts of 'General Interest 2019' with tie in 2017:\n", merged_table[merged_table['2017 Tie'] == 1]['General Interest 2019'].value_counts())

# Value counts of 'General Interest 2019' when there is no tie in 2017
print("Value counts of 'General Interest 2019' without tie in 2017:\n", merged_table[merged_table['2017 Tie'] == 0]['General Interest 2019'].value_counts())


Value counts of 'General Interest 2017' with tie in 2017:
 1    489
0    189
Name: General Interest 2017, dtype: int64
Value counts of 'General Interest 2017' without tie in 2017:
 1    1725109
0     465584
Name: General Interest 2017, dtype: int64
Value counts of 'General Interest 2017' with tie in 2019:
 1    6
0    1
Name: General Interest 2017, dtype: int64
Value counts of 'General Interest 2017' without tie in 2019:
 1    1725592
0     465772
Name: General Interest 2017, dtype: int64
Value counts of 'General Interest 2019' with tie in 2017:
 1    583
0     95
Name: General Interest 2019, dtype: int64
Value counts of 'General Interest 2019' without tie in 2017:
 1    2114679
0      76014
Name: General Interest 2019, dtype: int64


In [63]:
# Perform chi-square tests
from scipy.stats import chi2_contingency 

# First chi-square test
observed_1 = [[489, 189], [1725109, 465584]] 
chi2_1, p_1, dof_1, expected_1 = chi2_contingency(observed_1)

# Print results
print("Chi-square statistic (Test 1):", chi2_1)
print("P-value (Test 1):", p_1)
print("Degrees of freedom (Test 1):", dof_1)
print("Expected frequencies table (Test 1):")
print(expected_1)
print("\n")

# Second chi-square test
observed_2 = [[6, 1], [1725592, 465772]] 
chi2_2, p_2, dof_2, expected_2 = chi2_contingency(observed_2)


print("Chi-square statistic (Test 2):", chi2_2)
print("P-value (Test 2):", p_2)
print("Degrees of freedom (Test 2):", dof_2)
print("Expected frequencies table (Test 2):")
print(expected_2)
print("\n")

# Third chi-square test
observed_3 = [[583, 95], [2114679, 76014]] 
chi2_3, p_3, dof_3, expected_3 = chi2_contingency(observed_3)

print("Chi-square statistic (Test 3):", chi2_3)
print("P-value (Test 3):", p_3)
print("Degrees of freedom (Test 3):", dof_3)
print("Expected frequencies table (Test 3):")
print(expected_3)
print("\n")

Chi-square statistic (Test 1): 17.371286521779087
P-value (Test 1): 3.0743554227997474e-05
Degrees of freedom (Test 1): 1
Expected frequencies table (Test 1):
[[5.33891999e+02 1.44108001e+02]
 [1.72506411e+06 4.65628892e+05]]


Chi-square statistic (Test 2): 0.0
P-value (Test 2): 1.0
Degrees of freedom (Test 2): 1
Expected frequencies table (Test 2):
[[5.51215928e+00 1.48784072e+00]
 [1.72559249e+06 4.65771512e+05]]


Chi-square statistic (Test 3): 221.5483001746039
P-value (Test 3): 4.155900132350048e-50
Degrees of freedom (Test 3): 1
Expected frequencies table (Test 3):
[[6.54452229e+02 2.35477708e+01]
 [2.11460755e+06 7.60854522e+04]]


